# Data Cleaning & Preprocessing 

## Objective
1. Fix the date format
2. Create useful new columns (month, day of week)
3. Save clean data for analysis


---

In [14]:
# Import libraries
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

print(" Libraries imported!")

 Libraries imported!


In [15]:
# Function to load all CSV files from a folder
def load_all_csvs(folder_path):
    all_data = []
    all_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            all_data.append(df)
    return pd.concat(all_data, ignore_index=True)

# Load all 3 datasets
print(" Loading datasets...")
enrolment_df = load_all_csvs('../dataset/api_data_aadhar_enrolment')
demographic_df = load_all_csvs('../dataset/api_data_aadhar_demographic')
biometric_df = load_all_csvs('../dataset/api_data_aadhar_biometric')

print(f" Enrolment: {len(enrolment_df):,} rows")
print(f" Demographic: {len(demographic_df):,} rows")
print(f" Biometric: {len(biometric_df):,} rows")

 Loading datasets...
 Enrolment: 1,006,029 rows
 Demographic: 2,071,700 rows
 Biometric: 1,861,108 rows


## Fix Date Format

- Currently: dates are stored as text "01-03-2025"
- We need: dates as actual dates that Python understands

In [16]:
# Look at current date format
print("🔍 Current date format examples:")
print(f"Enrolment: {enrolment_df['date'].iloc[0]}")
print(f"Demographic: {demographic_df['date'].iloc[0]}")
print(f"Biometric: {biometric_df['date'].iloc[0]}")
print(f"\n Data type: {enrolment_df['date'].dtype}")

🔍 Current date format examples:
Enrolment: 02-03-2025
Demographic: 01-03-2025
Biometric: 01-03-2025

 Data type: object


In [17]:
# Convert date column to datetime format
# Format: DD-MM-YYYY (day-month-year)

print("🔄 Converting dates...")

# Convert each dataset's date column
enrolment_df['date'] = pd.to_datetime(enrolment_df['date'], format='%d-%m-%Y')
demographic_df['date'] = pd.to_datetime(demographic_df['date'], format='%d-%m-%Y')
biometric_df['date'] = pd.to_datetime(biometric_df['date'], format='%d-%m-%Y')

print(" Dates converted!")
print(f"\n New data type: {enrolment_df['date'].dtype}")
print(f"\n🔍 Example: {enrolment_df['date'].iloc[0]}")

🔄 Converting dates...
 Dates converted!

 New data type: datetime64[ns]

🔍 Example: 2025-03-02 00:00:00


### Create Useful New Columns

- **year** - Which year (2025)
- **month** - Month number (1-12)
- **month_name** - Month name (January, February, etc.)
- **day** - Day of month (1-31)
- **day_of_week** - Day number (0=Monday, 6=Sunday)
- **day_name** - Day name (Monday, Tuesday, etc.)
- **is_weekend** - True if Saturday or Sunday

In [18]:
# Function to add date-related columns to a dataframe
def add_date_columns(df):
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['month_name'] = df['date'].dt.month_name()
    df['day'] = df['date'].dt.day
    df['day_of_week'] = df['date'].dt.dayofweek  # 0=Monday, 6=Sunday
    df['day_name'] = df['date'].dt.day_name()
    df['is_weekend'] = df['day_of_week'].isin([5, 6])  # 5=Saturday, 6=Sunday
    
    return df

print("Function created! Applying to all datasets...")

Function created! Applying to all datasets...


In [19]:
# Apply to all 3 datasets
print("🔄 Adding date columns to ENROLMENT data...")
enrolment_df = add_date_columns(enrolment_df)

print("🔄 Adding date columns to DEMOGRAPHIC data...")
demographic_df = add_date_columns(demographic_df)

print("🔄 Adding date columns to BIOMETRIC data...")
biometric_df = add_date_columns(biometric_df)

print("\n All date columns added!")

🔄 Adding date columns to ENROLMENT data...
🔄 Adding date columns to DEMOGRAPHIC data...
🔄 Adding date columns to BIOMETRIC data...

 All date columns added!


In [20]:
# Let's see the new columns!
print(" NEW COLUMNS IN ENROLMENT DATA:")
print(enrolment_df.columns.tolist())

print("\n Sample data with new columns:")
enrolment_df[['date', 'month_name', 'day_name', 'is_weekend']].head(10)

 NEW COLUMNS IN ENROLMENT DATA:
['date', 'state', 'district', 'pincode', 'age_0_5', 'age_5_17', 'age_18_greater', 'year', 'month', 'month_name', 'day', 'day_of_week', 'day_name', 'is_weekend']

 Sample data with new columns:


,date,month_name,day_name,is_weekend
0,2025-03-02,March,Sunday,True
1,2025-03-09,March,Sunday,True
2,2025-03-09,March,Sunday,True
3,2025-03-09,March,Sunday,True
4,2025-03-09,March,Sunday,True
5,2025-03-09,March,Sunday,True
6,2025-03-09,March,Sunday,True
7,2025-03-09,March,Sunday,True
8,2025-03-09,March,Sunday,True
9,2025-03-09,March,Sunday,True


### Add Total Columns

- **total_enrolled** - Sum of all age groups for enrolment
- **total_demo_updates** - Sum of demographic updates
- **total_bio_updates** - Sum of biometric updates

In [21]:
# Add total column to each dataset

# For Enrolment: add all age groups
enrolment_df['total_enrolled'] = (
    enrolment_df['age_0_5'] + 
    enrolment_df['age_5_17'] + 
    enrolment_df['age_18_greater']
)

# For Demographic: add both age groups
demographic_df['total_updates'] = (
    demographic_df['demo_age_5_17'] + 
    demographic_df['demo_age_17_']
)

# For Biometric: add both age groups
biometric_df['total_updates'] = (
    biometric_df['bio_age_5_17'] + 
    biometric_df['bio_age_17_']
)

print(" Total columns added!")
print(f"\n Total Enrolments: {enrolment_df['total_enrolled'].sum():,}")
print(f" Total Demographic Updates: {demographic_df['total_updates'].sum():,}")
print(f" Total Biometric Updates: {biometric_df['total_updates'].sum():,}")

 Total columns added!

 Total Enrolments: 5,435,702
 Total Demographic Updates: 49,295,187
 Total Biometric Updates: 69,763,095


### Check State Names

Checking if there are any inconsistencies in state names (like "UP" vs "Uttar Pradesh").

In [22]:
# Get all unique state names
all_states_enrol = set(enrolment_df['state'].unique())
all_states_demo = set(demographic_df['state'].unique())
all_states_bio = set(biometric_df['state'].unique())

print(f" Unique states in Enrolment: {len(all_states_enrol)}")
print(f" Unique states in Demographic: {len(all_states_demo)}")
print(f" Unique states in Biometric: {len(all_states_bio)}")

# Find states that are in one dataset but not others
common_states = all_states_enrol & all_states_demo & all_states_bio
print(f"\n Common states in all 3 datasets: {len(common_states)}")

# Show extra states in demographic (since it has most - 65)
extra_demo = all_states_demo - common_states
if extra_demo:
    print(f"\n⚠ States only in Demographic data:")
    for state in sorted(extra_demo):
        print(f"   - {state}")

 Unique states in Enrolment: 55
 Unique states in Demographic: 65
 Unique states in Biometric: 57

 Common states in all 3 datasets: 52

⚠ States only in Demographic data:
   - 100000
   - BALANAGAR
   - Chhatisgarh
   - Darbhanga
   - Jaipur
   - Madanapalle
   - Nagpur
   - Puttenahalli
   - Raja Annamalai Puram
   - Uttaranchal
   - West Bengli
   - odisha
   - west Bengal


In [23]:
# Let's see all unique states in alphabetical order
print(" ALL STATES/UTs IN ENROLMENT DATA:")
print("="*60)
for i, state in enumerate(sorted(enrolment_df['state'].unique()), 1):
    print(f"{i:3}. {state}")

 ALL STATES/UTs IN ENROLMENT DATA:
  1. 100000
  2. Andaman & Nicobar Islands
  3. Andaman and Nicobar Islands
  4. Andhra Pradesh
  5. Arunachal Pradesh
  6. Assam
  7. Bihar
  8. Chandigarh
  9. Chhattisgarh
 10. Dadra & Nagar Haveli
 11. Dadra and Nagar Haveli
 12. Dadra and Nagar Haveli and Daman and Diu
 13. Daman & Diu
 14. Daman and Diu
 15. Delhi
 16. Goa
 17. Gujarat
 18. Haryana
 19. Himachal Pradesh
 20. Jammu & Kashmir
 21. Jammu And Kashmir
 22. Jammu and Kashmir
 23. Jharkhand
 24. Karnataka
 25. Kerala
 26. Ladakh
 27. Lakshadweep
 28. Madhya Pradesh
 29. Maharashtra
 30. Manipur
 31. Meghalaya
 32. Mizoram
 33. Nagaland
 34. ODISHA
 35. Odisha
 36. Orissa
 37. Pondicherry
 38. Puducherry
 39. Punjab
 40. Rajasthan
 41. Sikkim
 42. Tamil Nadu
 43. Telangana
 44. The Dadra And Nagar Haveli And Daman And Diu
 45. Tripura
 46. Uttar Pradesh
 47. Uttarakhand
 48. WEST BENGAL
 49. WESTBENGAL
 50. West  Bengal
 51. West Bangal
 52. West Bengal
 53. West bengal
 54. Westbengal


## Data Summary

Let's see our cleaned data summary.

In [24]:
# Summary of all datasets after cleaning
print("="*60)
print(" CLEANED DATA SUMMARY")
print("="*60)

print("\n ENROLMENT DATA:")
print(f"   Rows: {len(enrolment_df):,}")
print(f"   Columns: {len(enrolment_df.columns)}")
print(f"   Date Range: {enrolment_df['date'].min().strftime('%d %B %Y')} to {enrolment_df['date'].max().strftime('%d %B %Y')}")
print(f"   Total Enrolments: {enrolment_df['total_enrolled'].sum():,}")

print("\n DEMOGRAPHIC UPDATE DATA:")
print(f"   Rows: {len(demographic_df):,}")
print(f"   Columns: {len(demographic_df.columns)}")
print(f"   Date Range: {demographic_df['date'].min().strftime('%d %B %Y')} to {demographic_df['date'].max().strftime('%d %B %Y')}")
print(f"   Total Updates: {demographic_df['total_updates'].sum():,}")

print("\n BIOMETRIC UPDATE DATA:")
print(f"   Rows: {len(biometric_df):,}")
print(f"   Columns: {len(biometric_df.columns)}")
print(f"   Date Range: {biometric_df['date'].min().strftime('%d %B %Y')} to {biometric_df['date'].max().strftime('%d %B %Y')}")
print(f"   Total Updates: {biometric_df['total_updates'].sum():,}")

 CLEANED DATA SUMMARY

 ENROLMENT DATA:
   Rows: 1,006,029
   Columns: 15
   Date Range: 02 March 2025 to 31 December 2025
   Total Enrolments: 5,435,702

 DEMOGRAPHIC UPDATE DATA:
   Rows: 2,071,700
   Columns: 14
   Date Range: 01 March 2025 to 29 December 2025
   Total Updates: 49,295,187

 BIOMETRIC UPDATE DATA:
   Rows: 1,861,108
   Columns: 14
   Date Range: 01 March 2025 to 29 December 2025
   Total Updates: 69,763,095


In [25]:
# Show final structure of each dataframe
print(" FINAL ENROLMENT DATA STRUCTURE:")
enrolment_df.info()

 FINAL ENROLMENT DATA STRUCTURE:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006029 entries, 0 to 1006028
Data columns (total 15 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   date            1006029 non-null  datetime64[ns]
 1   state           1006029 non-null  object        
 2   district        1006029 non-null  object        
 3   pincode         1006029 non-null  int64         
 4   age_0_5         1006029 non-null  int64         
 5   age_5_17        1006029 non-null  int64         
 6   age_18_greater  1006029 non-null  int64         
 7   year            1006029 non-null  int32         
 8   month           1006029 non-null  int32         
 9   month_name      1006029 non-null  object        
 10  day             1006029 non-null  int32         
 11  day_of_week     1006029 non-null  int32         
 12  day_name        1006029 non-null  object        
 13  is_weekend      1006029 non-null  bool 

### Save Cleaned Data

In [26]:
# Create a processed_data folder if it doesn't exist
import os

processed_folder = '../processed_data'
if not os.path.exists(processed_folder):
    os.makedirs(processed_folder)
    print(f" Created folder: {processed_folder}")

# Save cleaned datasets
print(" Saving cleaned datasets...")

enrolment_df.to_csv(f'{processed_folder}/enrolment_cleaned.csv', index=False)
print(" Saved: enrolment_cleaned.csv")

demographic_df.to_csv(f'{processed_folder}/demographic_cleaned.csv', index=False)
print(" Saved: demographic_cleaned.csv")

biometric_df.to_csv(f'{processed_folder}/biometric_cleaned.csv', index=False)
print(" Saved: biometric_cleaned.csv")

print("\n All cleaned data saved successfully!")

 Saving cleaned datasets...
 Saved: enrolment_cleaned.csv
 Saved: demographic_cleaned.csv
 Saved: biometric_cleaned.csv

 All cleaned data saved successfully!
